## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

### Create an external table

In [3]:
# Examine DBFS root
display(dbutils.fs.ls('dbfs:/'))

In [4]:
%sql

-- Create a database
CREATE DATABASE IF NOT EXISTS wine_db

Import data with UI as described in the article and then follow the code below.

In [6]:
# Access the data in the table with Spark API

query = "select * from winequality_red"
wine = spark.sql(query)
display(wine)

In [7]:
%sql

-- Access the data in the table with Spark SQL
SELECT * FROM winequality_red

### Create a Delta Table

In [9]:
# File location and type
# file_location gets automatically generated and it can differ between different uploads. If you are running this code, you need to change accordingly. 
file_location = "/FileStore/tables/winequality_red-42ff5.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [10]:
# Create a view or table

temp_table_name = "winequality_red_view"
df.createOrReplaceTempView(temp_table_name)

In [11]:
%sql

/* Query the created temp table in a SQL cell */

SELECT * FROM winequality_red_view

In [12]:
# Replace whitespaces in column names with '_'

from pyspark.sql.functions import col

df_corr = df.select(col('fixed acidity').alias('fixed_acidity'), col('volatile acidity').alias('volatile_acidity'), \
                    col('citric acid').alias('citric_acid'), col('residual sugar').alias('residual_sugar'), col('chlorides'), \
                    col('free sulfur dioxide').alias('free_sulfur_dioxide'), col('total sulfur dioxide').alias('total_sulfur_dioxide'), \
                    col('density'), col('pH'), col('sulphates'), col('alcohol'), col('quality'))

display(df_corr)

In [13]:
# A temp view is only available to this particular notebook. If you'd like other users to be able to query this table, you can also create a Delta Table from the DataFrame. Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.

permanent_table_name = "wine_quality_delta"
df_corr.write.format("delta").saveAsTable(permanent_table_name)

In [14]:
# Databases and global tables in Hive 

display(dbutils.fs.ls('user/hive/warehouse'))

In [15]:
%sql
-- Update the Delta table

UPDATE wine_quality_delta SET citric_acid = 0 WHERE citric_acid < 0.1

In [16]:
%sql
-- Check modified values 

SELECT * FROM wine_quality_delta

In [17]:
%sql
-- Delete rows from the Delta table

DELETE FROM wine_quality_delta WHERE citric_acid = 0

In [18]:
%sql
-- Check new state of the table 

SELECT * FROM wine_quality_delta

In [19]:
%sql
-- Look at the history of modifications

DESCRIBE HISTORY wine_quality_delta

In [20]:
%sql
-- Read version 1

SELECT * FROM wine_quality_delta VERSION AS OF 1

--or (correct the value of the timestamp)
--SELECT * FROM wine_quality_delta TIMESTAMP AS OF '2020-04-09T15:20:11.000+0000'